1. EDA

In [148]:
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from pymc_marketing.mmm import MMM, GeometricAdstock, LogisticSaturation
from pymc_marketing.mmm.transformers import geometric_adstock, logistic_saturation
from pymc_marketing.prior import Prior

import pytimetk as tk

warnings.filterwarnings("ignore", category=FutureWarning)

az.style.use("arviz-darkgrid")
plt.rcParams["figure.figsize"] = [12, 7]
plt.rcParams["figure.dpi"] = 100

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = "retina"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [149]:
data=pd.read_csv("data.csv")
data.columns.to_list()

['date',
 'trend',
 'seasonality',
 'demand',
 'demand_proxy',
 'tv_spend_raw',
 'ooh_spend_raw',
 'print_spend_raw',
 'facebook_spend_raw',
 'search_spend_raw',
 'tv_spend_scaled',
 'ooh_spend_scaled',
 'print_spend_scaled',
 'facebook_spend_scaled',
 'search_spend_scaled',
 'ooh_adstock',
 'tv_adstock',
 'print_adstock',
 'facebook_adstock',
 'search_adstock',
 'ooh_saturated',
 'tv_saturated',
 'print_saturated',
 'facebook_saturated',
 'search_saturated',
 'facebook_I',
 'search_clicks_P',
 'competitor_sales_B',
 'competitor_sales_B_scaled',
 'facebook_I_scaled',
 'search_clicks_P_scaled',
 'sales',
 'ooh_sales',
 'tv_sales',
 'print_sales',
 'facebook_sales',
 'search_sales',
 'epsilon']

Reducing our dataset to a one a company could give us from raw data

In [150]:
df=data.copy()
df=df[["date","sales","tv_spend_raw","ooh_spend_raw","print_spend_raw","facebook_I","search_clicks_P","search_spend_raw","competitor_sales_B","facebook_spend_raw"]]
df["date"] = pd.to_datetime(df["date"])
df

,date,sales,tv_spend_raw,ooh_spend_raw,print_spend_raw,facebook_I,search_clicks_P,search_spend_raw,competitor_sales_B,facebook_spend_raw
0,2021-01-04,162277.109282,0.0,963.639807,0.000000,3.713912e+06,198584.891345,2182.363211,141967.828958,4296.052070
1,2021-01-11,170493.217138,0.0,1015.604279,0.000000,3.589720e+06,130288.368188,2151.854375,133677.920909,4324.637643
2,2021-01-18,144523.455074,0.0,1102.396049,0.000000,4.789362e+06,216728.973534,2168.730601,162910.166078,4926.757799
3,2021-01-25,239399.578756,0.0,1371.427530,0.000000,8.489669e+06,423877.974468,3306.440497,113291.431377,7538.774028
4,2021-02-01,195422.511307,0.0,1015.958803,0.000000,4.553082e+06,216162.619678,2378.653779,157950.654088,5212.689979
...,...,...,...,...,...,...,...,...,...,...
204,2024-12-02,291851.877398,0.0,1590.686057,991.244062,1.233491e+07,360474.711439,3682.570068,335040.024962,11355.041460
205,2024-12-09,286104.946753,0.0,2129.837419,0.000000,9.289202e+06,345847.627448,3834.050375,437793.761262,10681.589975
206,2024-12-16,323712.054120,0.0,1821.433848,0.000000,7.710938e+06,302433.929114,3064.260002,266640.978754,9576.761963
207,2024-12-23,355519.774311,0.0,1937.734973,0.000000,1.238454e+07,505206.667570,3851.515233,281461.313125,13402.434707


In [151]:
channels = ["tv","ooh", "print", "facebook", "search"]
for i in channels:
  df.rename(columns={f"{i}_spend_raw":f"{i}_s"}, inplace=True)
  #df[f"{i}_s"]*=100000
df.columns.to_list() 

['date',
 'sales',
 'tv_s',
 'ooh_s',
 'print_s',
 'facebook_I',
 'search_clicks_P',
 'search_s',
 'competitor_sales_B',
 'facebook_s']

In [158]:
# Visualize Sales and Spend by Channel
df\
  .melt(id_vars=["date"], 
  value_vars=["sales", "tv_s", "ooh_s", "print_s", "facebook_s", "search_s","competitor_sales_B","facebook_I", "search_clicks_P"])\
    .groupby("variable")\
      .plot_timeseries(
        "date",
        "value",
        color_column="variable",
        facet_ncol=3,
        width=800,
        height=600,
        legend_show=False,)

In [153]:
# Total Spend and Revenue Analysis
total_spend = df[["tv_s", "ooh_s", "print_s", "facebook_s", "search_s"]].sum(axis=0).sum()
total_revenue = df["sales"].sum()

print(f"Total Spend: {total_spend:.2f}")
print(f"Total Revenue: {total_revenue:.2f}")
print(f"Revenue/Spend: {total_revenue / total_spend}")

Total Spend: 3836196.94
Total Revenue: 56452678.35
Revenue/Spend: 14.71579253742949


In [154]:
# Monthly Ad Spend Analysis
df[["tv_s", "ooh_s", "print_s", "facebook_s", "search_s"]].describe().apply(lambda y:y.apply(lambda x:"${:,.0f}".format(x)))

,tv_s,ooh_s,print_s,facebook_s,search_s
count,$209,$209,$209,$209,$209
mean,"$6,342","$1,657",$142,"$6,886","$3,328"
std,"$11,416",$436,$476,"$2,470",$809
min,$0,$810,$0,"$2,052","$1,589"
25%,$0,"$1,326",$0,"$4,927","$2,673"
50%,$0,"$1,611",$0,"$6,642","$3,322"
75%,"$12,002","$1,959",$0,"$8,313","$3,834"
max,"$49,311","$2,779","$2,839","$15,080","$5,387"


2. FE

In [160]:
df_features=df.assign(
  year=lambda x: x["date"].dt.year,
  month=lambda x: x["date"].dt.month,
  dayofyear=lambda x: x["date"].dt.dayofyear,
).assign(trend=lambda x: df.index)
df_features

,date,sales,tv_s,ooh_s,print_s,facebook_I,search_clicks_P,search_s,competitor_sales_B,facebook_s,year,month,dayofyear,trend
0,2021-01-04,162277.109282,0.0,963.639807,0.000000,3.713912e+06,198584.891345,2182.363211,141967.828958,4296.052070,2021,1,4,0
1,2021-01-11,170493.217138,0.0,1015.604279,0.000000,3.589720e+06,130288.368188,2151.854375,133677.920909,4324.637643,2021,1,11,1
2,2021-01-18,144523.455074,0.0,1102.396049,0.000000,4.789362e+06,216728.973534,2168.730601,162910.166078,4926.757799,2021,1,18,2
3,2021-01-25,239399.578756,0.0,1371.427530,0.000000,8.489669e+06,423877.974468,3306.440497,113291.431377,7538.774028,2021,1,25,3
4,2021-02-01,195422.511307,0.0,1015.958803,0.000000,4.553082e+06,216162.619678,2378.653779,157950.654088,5212.689979,2021,2,32,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,2024-12-02,291851.877398,0.0,1590.686057,991.244062,1.233491e+07,360474.711439,3682.570068,335040.024962,11355.041460,2024,12,337,204
205,2024-12-09,286104.946753,0.0,2129.837419,0.000000,9.289202e+06,345847.627448,3834.050375,437793.761262,10681.589975,2024,12,344,205
206,2024-12-16,323712.054120,0.0,1821.433848,0.000000,7.710938e+06,302433.929114,3064.260002,266640.978754,9576.761963,2024,12,351,206
207,2024-12-23,355519.774311,0.0,1937.734973,0.000000,1.238454e+07,505206.667570,3851.515233,281461.313125,13402.434707,2024,12,358,207


In [162]:
df_features=df_features[['date', 'sales', 'tv_s', 'ooh_s', 'print_s', 'facebook_s', 'search_s','trend','year','month','dayofyear']]
df_features.head(5)

,date,sales,tv_s,ooh_s,print_s,facebook_s,search_s,trend,year,month,dayofyear
0,2021-01-04,162277.109282,0.0,963.639807,0.0,4296.052070,2182.363211,0,2021,1,4
1,2021-01-11,170493.217138,0.0,1015.604279,0.0,4324.637643,2151.854375,1,2021,1,11
2,2021-01-18,144523.455074,0.0,1102.396049,0.0,4926.757799,2168.730601,2,2021,1,18
3,2021-01-25,239399.578756,0.0,1371.427530,0.0,7538.774028,3306.440497,3,2021,1,25
4,2021-02-01,195422.511307,0.0,1015.958803,0.0,5212.689979,2378.653779,4,2021,2,32
